In [1]:
import os
os.chdir("../tsp")
print("Current Working Directory:", os.getcwd())

import time
import torch
from torch.distributions import Categorical, kl
# from d2l.torch import Animator

from net import Net
from aco import ACO
from utils import gen_pyg_data, load_test_dataset
from greedy import test_greedy, GreedyTSP
from tqdm import tqdm

torch.manual_seed(12345)

EPS = 1e-10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Current Working Directory: /Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/tsp
Using device: cpu


In [2]:
import os

TSPLIB_DIR = "../data/tsp/TSPLIB"
tsp_files = [f for f in os.listdir(TSPLIB_DIR) if f.endswith(".tsp")]
print(tsp_files)

['pr439.tsp', 'rd100.tsp', 'rl5934.tsp', 'pcb442.tsp', 'u2319.tsp', 'gil262.tsp', 'pcb3038.tsp', 'lin105.tsp', 'fl417.tsp', 'tsp225.tsp', 'fl1400.tsp', 'nrw1379.tsp', 'd2103.tsp', 'kroA150.tsp', 'pcb1173.tsp', 'd198.tsp', 'fl1577.tsp', 'ch130.tsp', 'kroB100.tsp', 'u1060.tsp', 'berlin52.tsp', 'eil51.tsp', 'rl1304.tsp', 'u2152.tsp', 'u724.tsp', 'kroD100.tsp', 'pr299.tsp', 'rd400.tsp', 'vm1084.tsp', 'rat575.tsp', 'd1655.tsp', 'ch150.tsp', 'd15112.tsp', 'pr107.tsp', 'kroB200.tsp', 'brd14051.tsp', 'a280.tsp', 'd1291.tsp', 'pr264.tsp', 'pr76.tsp', 'd493.tsp', 'pr136.tsp', 'rat195.tsp', 'rl11849.tsp', 'kroA100.tsp', 'kroB150.tsp', 'bier127.tsp', 'kroC100.tsp', 'usa13509.tsp', 'eil76.tsp', 'pr124.tsp', 'rl1323.tsp', 'p654.tsp', 'rl1889.tsp', 'd657.tsp', 'eil101.tsp', 'fnl4461.tsp', 'pr2392.tsp', 'rat783.tsp', 'ts225.tsp', 'u1432.tsp', 'u1817.tsp', 'lin318.tsp', 'd18512.tsp', 'rl5915.tsp', 'st70.tsp', 'rat99.tsp', 'fl3795.tsp', 'u159.tsp', 'kroA200.tsp', 'u574.tsp', 'pr1002.tsp', 'pr152.tsp', '

In [2]:
@torch.no_grad()
def infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse=None):
    if model:
        model.eval()
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO(
        n_ants=n_ants,
        heuristic=heu_mat,
        distances=distances,
        device=device
        )
    
    else:
        aco = ACO(
        n_ants=n_ants,
        distances=distances,
        device=device
        )
        if k_sparse:
            aco.sparsify(k_sparse)
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results

@torch.no_grad()
def test(dataset, model, n_ants, t_aco, k_sparse=None, verbose=False):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    if verbose:
        for pyg_data, distances in tqdm(dataset, desc="Testing Deep ACO Algorithm"):
            results = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse)
            sum_results += results
    else:
        for pyg_data, distances in dataset:
            results = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse)
            sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start


In [ ]:
torch.set_printoptions(precision=3, sci_mode=False)
# Configuration
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)

# # Testing Deep ACO Model
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('Deep ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

In [80]:
# Testing Greedy Algorithm
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=True)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")

GreedyTSP is using: cpu


Testing Greedy Algorithm: 100%|██████████| 1280/1280 [00:00<00:00, 6440.67it/s]

--------Greedy Algorithm:--------
Total duration:  0.20189809799194336
Average cost is 4.539401644987083.


# Across Scale

In [5]:
torch.set_printoptions(precision=3, sci_mode=False)
# Configuration
pretrain_n_node = 20
n_ants = 20
test_n_node = 100
k_sparse = test_n_node-1
t_aco = [1, 20, 50,]
test_list = load_test_dataset(test_n_node, k_sparse, device)

# # Testing Deep ACO Model
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{pretrain_n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('Deep ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

/Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/tsp/utils.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_tensor = torch.load(f'../data/tsp/testDataset

Deep ACO Model:
Total duration:  487.3229932785034
T=1, average cost is 10.886425971984863.
T=20, average cost is 9.681572914123535.
T=50, average cost is 9.40514087677002.


In [6]:
# Testing Greedy Algorithm
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=True)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")

GreedyTSP is using: cpu


Testing Greedy Algorithm: 100%|██████████| 1280/1280 [00:00<00:00, 1348.60it/s]

--------Greedy Algorithm:--------
Total duration:  0.9518709182739258
Average cost is 9.67953964085406.


# Test TSPLIB

In [3]:
from utils import gen_pyg_data_fully_connected, load_TSPLIB_test_instance

TSPLIB_DIR = '../data/tsp/TSPLIB'

file_names=['bier127.tsp', 'ch130.tsp', 'ch150.tsp', 'eil101.tsp']
k_sparse = 20
tsplib_test_list = load_TSPLIB_test_instance(dir=TSPLIB_DIR, file_names=file_names, device=device, k_sparse = k_sparse, normalized=True)

torch.set_printoptions(precision=3, sci_mode=False)
# Configuration
pretrain_n_node = 100
n_ants = 200
t_aco = [1, 2, 10]

k_sparse = 20

# Testing ACO Model
avg_aco_best, duration = test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

# Testing Deep ACO Model
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{pretrain_n_node}.pt', map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_tsp, n_ants, t_aco, k_sparse, verbose=True)
print('Deep ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(tsplib_test_list, greedy_tsp, show_progress=True)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")

/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_41695/2697478731.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_tsp.load_state_dict(torch.load(f'../pret

ACO Model:
Total duration:  2.243971824645996
T=1, average cost is 15.53651237487793.
T=2, average cost is 14.156938552856445.
T=10, average cost is 11.681442260742188.


Testing Deep ACO Algorithm: 100%|██████████| 4/4 [00:02<00:00,  1.81it/s]


Deep ACO Model:
Total duration:  2.218463897705078
T=1, average cost is 9.516571044921875.
T=2, average cost is 9.178625106811523.
T=10, average cost is 8.79952621459961.
GreedyTSP is using: cpu


Testing Greedy Algorithm: 100%|██████████| 4/4 [00:00<00:00, 661.82it/s]

--------Greedy Algorithm:--------
Total duration:  0.007444858551025391
Average cost is 10.27803460357245.


In [4]:
from utils import gen_pyg_data_fully_connected, load_TSPLIB_test_instance

TSPLIB_DIR = '../data/tsp/TSPLIB'

file_names=['bier127.tsp', 'ch130.tsp', 'ch150.tsp', 'eil101.tsp']
k_sparse = 20
tsplib_test_list = load_TSPLIB_test_instance(dir=TSPLIB_DIR, file_names=file_names, device=device, k_sparse = k_sparse, normalized=False)

torch.set_printoptions(precision=3, sci_mode=False)
# Configuration
pretrain_n_node = 100
n_ants = 200
t_aco = [1, 2, 10]

k_sparse = 20

# Testing ACO Model
avg_aco_best, duration = test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

# Testing Deep ACO Model
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{pretrain_n_node}.pt', map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_tsp, n_ants, t_aco, k_sparse, verbose=True)
print('Deep ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(tsplib_test_list, greedy_tsp, show_progress=True)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")

/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_41695/2318233027.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_tsp.load_state_dict(torch.load(f'../pret

ACO Model:
Total duration:  2.27939772605896
T=1, average cost is 55859.64453125.
T=2, average cost is 55737.6796875.
T=10, average cost is 54028.4453125.


Testing Deep ACO Algorithm: 100%|██████████| 4/4 [00:02<00:00,  1.81it/s]


Deep ACO Model:
Total duration:  2.2157368659973145
T=1, average cost is 169340.515625.
T=2, average cost is 161547.640625.
T=10, average cost is 160972.9375.
GreedyTSP is using: cpu


Testing Greedy Algorithm: 100%|██████████| 4/4 [00:00<00:00, 670.45it/s]

--------Greedy Algorithm:--------
Total duration:  0.007425785064697266
Average cost is 38086.73029972613.


## large n_node rl11849.tsp

In [5]:
from utils import gen_pyg_data_fully_connected, load_TSPLIB_test_instance

TSPLIB_DIR = '../data/tsp/TSPLIB'

file_names=['rl11849.tsp']
k_sparse = 50
tsplib_test_list = load_TSPLIB_test_instance(dir=TSPLIB_DIR, file_names=file_names, device=device, k_sparse = k_sparse, normalized=True)

torch.set_printoptions(precision=3, sci_mode=False)
# Configuration
pretrain_n_node = 500
n_ants = 50
# t_aco = [1, 2, 10]
t_aco = [1]

# Testing ACO Model
avg_aco_best, duration = test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

# Testing Deep ACO Model
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{pretrain_n_node}.pt', map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_tsp, n_ants, t_aco, k_sparse, verbose=True)
print('Deep ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(tsplib_test_list, greedy_tsp, show_progress=True)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")

/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_41695/2878881267.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_tsp.load_state_dict(torch.load(f'../pret

ACO Model:
Total duration:  103.54408288002014
T=1, average cost is 229.74734497070312.


Testing Deep ACO Algorithm: 100%|██████████| 1/1 [01:45<00:00, 105.78s/it]


Deep ACO Model:
Total duration:  105.781653881073
T=1, average cost is 159.37045288085938.
GreedyTSP is using: cpu


Testing Greedy Algorithm: 100%|██████████| 1/1 [00:00<00:00,  2.44it/s]

--------Greedy Algorithm:--------
Total duration:  0.41199207305908203
Average cost is 58.71736486419104.
